# Idea
---
1. [x] Quantity definitions:
    1. [x] Query all quantities
    2. [x] Query all applicable units of each quantity
    3. [x] Query all conversionMultipliers of each unit to determine the base unit  
        
2. [] Quantity hierarchy:
    1. [x] use attribute skos:broader => points to super-quantity (only available for some quantites) 
    2. [] cluster quantities according to hasDimensionVector: one or multiple cluster members may be super-quantities, need manual analysis
   
Overview: http://www.qudt.org/doc/DOC_VOCAB-QUANTITY-KINDS.html  
Sparql-Endpoint: http://www.qudt.org/fuseki/dataset.html?tab=query&ds=/qudt 
***

## 1. Quantity definitions

### A. Query all quantities

In [ ]:
from SPARQLWrapper import SPARQLWrapper, POST, BASIC, JSON

sparql = SPARQLWrapper("http://www.qudt.org/fuseki/qudt/sparql")
sparql.setMethod(POST)

sparql.setQuery("""
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?subject
    WHERE {
      ?subject  rdf:type qudt:QuantityKind.
    } 
    LIMIT 999
""")
sparql.setReturnFormat(JSON)
#print(sparql.query())
quantityKind = sparql.query().convert()



***show data results***

In [ ]:
for uri in quantityKind["results"]["bindings"]:
    print(uri)

In [ ]:
a=0

for uri in quantityKind["results"]["bindings"]:
    #print(uri["subject"]["value"])
    a=a+1

print(a)

***init nested dictionary, set URI of quantities as KEY & empty dictionary as value***

In [ ]:
# initializing dictionary
QuantityDictionary = {}

for uri in quantityKind["results"]["bindings"]:
    QuantityDictionary[uri["subject"]["value"]] = {}

# print whole dictionary 
print(QuantityDictionary)
#for quantity in QuantityDictionary:
#    print(quantity)

# Getting first key in dictionary
key1 = list(QuantityDictionary.keys())[0]

# printing initial key
print("The first key of dictionary is : " + key1)

res2 = list(QuantityDictionary.keys())[1]

# printing initial key
print("The 2nd key of dictionary is : " + res2)


In [ ]:
# update main dictionary with input
QuantityDictionary2 = {}

for uri in quantityKind["results"]["bindings"]:
    QuantityDictionary2[uri["subject"]["value"]] = {}

TestDict = {'val1','val2'}
TestDict2 = {'val3','val4'}
print(key1)
QuantityDictionary2.update({key1: TestDict})
#print(QuantityDictionary2)

for upd in QuantityDictionary2:
    #print(upd)
    QuantityDictionary2.update({upd: TestDict})
#car.update({"color": "White"})
#print(car) 
#print(QuantityDictionary2)
print(QuantityDictionary2.items())

### B. Query all applicable units of each quantity

***show applicalbe units of one quantity***

In [ ]:
sparql.setQuery("""
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?applicableUnit
    WHERE {
      <http://qudt.org/vocab/quantitykind/Time> qudt:applicableUnit ?applicableUnit.
    } 
    LIMIT 999
""")
sparql.setReturnFormat(JSON)
applicableUnitTest = sparql.query().convert()
print(applicableUnitTest)

In [ ]:
#print(results)
b=0

for uri in applicableUnitTest["results"]["bindings"]:
    print(uri["applicableUnit"]["value"])
    b=b+1
    


In [ ]:
# Anzahl applicableUnit's
print(b)

***example f string format for individual uri query***

In [ ]:
# test 1: build query to compare right format
testURI1 ="""
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?applicableUnit
    WHERE {
      <http://qudt.org/vocab/quantitykind/MicrobialFormation> qudt:applicableUnit ?applicableUnit.
    } 
    LIMIT 999
"""
test1 = testURI1
print(test1)

# test 2:  getting the query string via uri variable from dictionary
testURI2 = str(list(QuantityDictionary.keys())[0])
s = f"""
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?applicableUnit
    WHERE {{
      <{testURI2}> qudt:applicableUnit ?applicableUnit.
    }} 
    LIMIT 999
"""
test2 = f'{s}'
print(test2)

# compare result
if test1==test2:
    print('f string build successfully')
else: 
    print('error')
    

***define function to query of individual applicableUnit depending on its URI by f string & build dictionary***

In [ ]:
quantitiyURI = 'http://qudt.org/vocab/quantitykind/Time'

sparql.setQuery(f"""
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?applicableUnit
    WHERE {{
      <{quantitiyURI}> qudt:applicableUnit ?applicableUnit.
    }} 
    LIMIT 999
""")
sparql.setReturnFormat(JSON)
applicableUnits = sparql.query().convert()
print(applicableUnits)

In [ ]:
def GetApplicableUnits(quantitiyURI):
    sparql.setQuery(f"""
        PREFIX qudt: <http://qudt.org/schema/qudt/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT ?applicableUnit
        WHERE {{
          <{quantitiyURI}> qudt:applicableUnit ?applicableUnit.
        }} 
        LIMIT 999
    """)
    sparql.setReturnFormat(JSON)
    applicableUnitsRaw = sparql.query().convert()

    ApplicableUnitsDictionary = {}
    for unit in applicableUnitsRaw["results"]["bindings"]:
        ApplicableUnitsDictionary[unit["applicableUnit"]["value"]] = {}

    return ApplicableUnitsDictionary
    


***define function to query of individual applicableUnit depending on its URI by f string & build list***

In [ ]:
def GetApplicableUnitsList(quantitiyURI):
    sparql.setQuery(f"""
        PREFIX qudt: <http://qudt.org/schema/qudt/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT ?applicableUnit
        WHERE {{
          <{quantitiyURI}> qudt:applicableUnit ?applicableUnit.
        }} 
        LIMIT 999
    """)
    sparql.setReturnFormat(JSON)
    applicableUnitsRaw = sparql.query().convert()
    #print(applicableUnits)
    ApplicableUnitsList = []
    for unit in applicableUnitsRaw["results"]["bindings"]:
        ApplicableUnitsList.append([unit["applicableUnit"]["value"]])
        #print(ApplicableUnitsDictionary)
    return ApplicableUnitsList

***update QuantityDictionary***

In [ ]:
# fill with applicable units TAKES A FEW MINUTES
for key in QuantityDictionary:
    QuantityDictionary.update({key: GetApplicableUnits(key)})

In [ ]:
#print(QuantityDictionary)
#print(list(QuantityDictionary.values())[6].values())
#print(list(QuantityDictionary.values())[6].keys())

In [ ]:
# test do update another key value instance
for key in QuantityDictionary:
    QuantityDictionary.update({key: 'description'})
    


In [ ]:
print(QuantityDictionary)

## Überlegungen Dictionary Aufbau

In [ ]:

Quantity1 = {
    'quantityKind': [
        {'URI': 'http://qudt.org/vocab/quantitykind/Time', 'quantityName': 'Time', 'isSiUnit': True}
    ],
    'applicableUnits': [
        {'URI': 'http://qudt.org/vocab/unit/SEC', 'unitName': 'second', 'conversionMultiplier': 1, 'isBaseUnit': True},
        {'URI': 'http://qudt.org/vocab/unit/MilliPA-SEC-PER-BAR', 'unitName': 'second', 'conversionMultiplier': 1, 'baseUnit': False}
    ]
} 
print(Quantity1.keys()) 
    
Quantity2 = {
    'quantityKind': {'URI': 'http://qudt.org/vocab/quantitykind/Time', 'quantityName': 'Time', 'isSiUnit': True}
    ,
    'applicableUnits': [
        {'URI': 'http://qudt.org/vocab/unit/SEC', 'unitName': 'second', 'conversionMultiplier': 1, 'isBaseUnit': True},
        {'URI': 'http://qudt.org/vocab/unit/MilliPA-SEC-PER-BAR', 'unitName': 'second', 'conversionMultiplier': 1, 'baseUnit': False}
    ]
} 
print(Quantity2.keys()) 
Quantities = {
    'Quantity1': Quantity1,
    'Quantity2': Quantity2
}
#print(Quantity1.keys())
#print(Quantity1.values())
print(Quantities.values())

In [ ]:
# option 1: array of nested objects
DictionaryArray = []
QuantityInArray = {
    'quantityKind': {'URI': 'http://qudt.org/vocab/quantitykind/Time', 'quantityName': 'Time', 'isSiUnit': True},
    'applicableUnits': [
        {'URI': 'http://qudt.org/vocab/unit/SEC', 'unitName': 'second', 'conversionMultiplier': 1, 'isBaseUnit': True},
        {'URI': 'http://qudt.org/vocab/unit/MilliPA-SEC-PER-BAR', 'unitName': 'second', 'conversionMultiplier': 1, 'baseUnit': False}
    ]
} 
# --> fill array with quantities

In [ ]:
# option 2: nestet object of nested objects with unit array of objects
QuantitiesDict = {
    'quantityKind': { 
        'ontology_iri': 'http://qudt.org/vocab/quantitykind/Mass',
        'description': 'In physics, mass, more specifically inertial mass, can be defined as a quantitative measure of an objects resistance to acceleration',
        'main_unit': 'kg',
        'isSiUnit': True,
        'applicableUnits': [
            {'ontology_iri': 'http://qudt.org/vocab/unit/GM', 'unit': 'g', 'factor': 1000, 'isBaseUnit': False},
            {'ontology_iri': 'http://qudt.org/vocab/unit/MilliGM', 'unit': 'mg', 'factor': 0.001, 'isBaseUnit': False},
        ]
    }
}


***show data results of one query***

In [ ]:
for uri in quantityKind["results"]["bindings"]:
    print(uri["subject"]["value"])

In [ ]:
b=0

for unit in applicableUnits["results"]["bindings"]:
    print(unit["applicableUnit"]["value"])
    b=b+1


***store unit data of one quantity into class***

## C. Query all conversionMultipliers of each unit to determine the base unit

In [ ]:
sparql.setQuery("""
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?conversionMultiplier ?ucumCode
    WHERE {
        <http://qudt.org/vocab/unit/MilliPA-SEC-PER-BAR> qudt:conversionMultiplier ?conversionMultiplier.
        <http://qudt.org/vocab/unit/MilliPA-SEC-PER-BAR> qudt:ucumCode ?ucumCode.
    } 
    LIMIT 999
""")
sparql.setReturnFormat(JSON)
quantityKind = sparql.query().convert()
#print(sparql.query())
print(quantityKind)